In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nisrinadinda","key":"3739f927160ceab79d4c5292bae2938f"}'}

In [ ]:
!rm -rf /root/.kaggle.
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle/kaggle.json

In [ ]:
!ls /root/.kaggle/kaggle.json

/root/.kaggle/kaggle.json


In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download thomasqazwsxedc/alphabet-characters-fonts-dataset

 99% 553M/559M [00:18<00:00, 29.4MB/s]
100% 559M/559M [00:18<00:00, 30.9MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!kaggle datasets download thomasqazwsxedc/alphabet-characters-fonts-dataset -f character_fonts(with handwritten data).npz

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `kaggle datasets download thomasqazwsxedc/alphabet-characters-fonts-dataset -f character_fonts(with handwritten data).npz'


In [ ]:
!unzip /content/alphabet-characters-fonts-dataset.zip

Streaming output truncated to the last 5000 lines.
  inflating: Images/Images/Z/5500.png  
  inflating: Images/Images/Z/5501.png  
  inflating: Images/Images/Z/5502.png  
  inflating: Images/Images/Z/5503.png  
  inflating: Images/Images/Z/5504.png  
  inflating: Images/Images/Z/5505.png  
  inflating: Images/Images/Z/5506.png  
  inflating: Images/Images/Z/5507.png  
  inflating: Images/Images/Z/5508.png  
  inflating: Images/Images/Z/5509.png  
  inflating: Images/Images/Z/551.png  
  inflating: Images/Images/Z/5510.png  
  inflating: Images/Images/Z/5511.png  
  inflating: Images/Images/Z/5512.png  
  inflating: Images/Images/Z/5513.png  
  inflating: Images/Images/Z/5514.png  
  inflating: Images/Images/Z/5515.png  
  inflating: Images/Images/Z/5516.png  
  inflating: Images/Images/Z/5517.png  
  inflating: Images/Images/Z/5518.png  
  inflating: Images/Images/Z/5519.png  
  inflating: Images/Images/Z/552.png  
  inflating: Images/Images/Z/5520.png  
  inflating: Images/Images/Z/55

In [ ]:
import pandas as pd
import numpy as np

## Load Kaggle Dataset

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%config InlineBackend.figure_format = 'svg' # using svg makes plots clearer
%matplotlib inline

In [ ]:
def load_kaggle_dataset():
  data = np.load('/content/character_fonts (with handwritten data).npz')
  return data['images'], data['labels']

In [ ]:
dataimg, datalabel = load_kaggle_dataset()

In [ ]:
dataimg.dtype

dtype('uint8')

In [ ]:
dataimg.dtype

dtype('uint8')

In [ ]:
print('Data images shape: {}'.format(dataimg.shape))
print('Data labels shape: {}'.format(datalabel.shape))

Data images shape: (762213, 28, 28)
Data labels shape: (762213,)


In [ ]:
dataimg[0].shape

(28, 28)

In [ ]:
datalabel[0].shape

()

In [ ]:
import matplotlib
matplotlib.use("Agg")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
from imutils import build_montages
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
dataimg = [cv2.resize(image, (32, 32)) for image in dataimg]
dataimg = np.array(dataimg, dtype="float32")
# add a channel dimension to every image in the dataset and scale the
# pixel intensities of the images from [0, 255] down to [0, 1]
dataimg = np.expand_dims(dataimg, axis=-1)
dataimg /= 255.0

In [ ]:
dataimg.shape

(762213, 32, 32, 1)

In [ ]:
datalabel = np.array(datalabel, dtype="int")

In [ ]:
datalabel.dtype

dtype('int64')

In [ ]:
lb = LabelBinarizer()
datalabel = lb.fit_transform(datalabel)
counts = datalabel.sum(axis=0)

In [ ]:
class_weight = {}
# loop over all classes and calculate the class weight
for i in range(0, len(counts)):
	class_weight[i] = counts.max() / counts[i]

In [ ]:
(X_train, X_test, y_train, y_test) = train_test_split(dataimg,
	datalabel, test_size=0.25, stratify=datalabel, random_state=42)

In [ ]:
EPOCHS = 7
INIT_LR = 1e-1
BS = 128

##Data Augmentation

In [ ]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(
	rotation_range=10,
	zoom_range=0.05,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.15,
	horizontal_flip=False,
	fill_mode="nearest")


## RNN Model

In [ ]:
pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 18.3 MB/s 


In [ ]:
import tensorflow_addons as tfa

In [ ]:
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, Reshape,MaxPool2D,Dense
from tensorflow.keras.layers import Bidirectional, LSTM, Lambda
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
import numpy as np
#from configuration import window_height, window_width, MPoolLayers_ALL, LastFilters, NUnits

import tensorflow as tf

import math
import cv2

import string

In [ ]:
from tensorflow.keras.layers import *

In [ ]:
char_list = string.ascii_letters+string.digits
l="0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [ ]:
inputs = Input(shape=(32,32,1))
convolution_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
pooling_1 = MaxPool2D(pool_size=(2, 2), strides=2)(convolution_1)
convolution_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pooling_1)
pooling_2 = MaxPool2D(pool_size=(2, 2), strides=2)(convolution_2)
convolution_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pooling_2)
pooling_3 = MaxPool2D(pool_size=(2, 2), strides=2)(convolution_3)
convolution_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pooling_3)
pooling_4 = MaxPool2D(pool_size=(2, 1))(convolution_4)
convolution_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pooling_4)
batch_norm_5 = BatchNormalization()(convolution_5)
convolution_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
pooling_6 = MaxPool2D(pool_size=(2, 1))(convolution_6)
batch_norm_6 = BatchNormalization()(pooling_6)
convolution_7 = Conv2D(512, (1,1), activation = 'relu')(batch_norm_6)
squeezed = tf.squeeze(convolution_7, axis=1)

# bidirectional LSTM
blstm_1 = Bidirectional(LSTM(256, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(256, return_sequences=True, dropout = 0.2))(blstm_1)

x = Flatten()(blstm_2)
x = Dropout(0.2)(x)
x = Dense(26, activation='relu')(x)
outputs = Activation("softmax")(x)

ocr_rnn = Model(inputs, outputs)

In [ ]:
print(ocr_rnn.summary())

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 32, 32, 1)]       0         
                                                                 
 conv2d_35 (Conv2D)          (None, 32, 32, 64)        640       
                                                                 
 max_pooling2d_25 (MaxPoolin  (None, 16, 16, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_36 (Conv2D)          (None, 16, 16, 128)       73856     
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 8, 8, 128)        0         
 g2D)                                                            
                                                                 
 conv2d_37 (Conv2D)          (None, 8, 8, 256)         2951

In [ ]:
opt = SGD(learning_rate=INIT_LR, decay=INIT_LR / EPOCHS)
ocr_rnn.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

##Fit the model

In [ ]:
class AccCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.998):
      self.model.stop_training = True

    if(logs.get('val_accuracy') > 0.95):
      self.model.stop_training = True

callback = AccCallback()

In [ ]:
print("[INFO] training network...")
H = ocr_rnn.fit(
	aug.flow(X_train, y_train, batch_size=BS),
	validation_data=(X_test, y_test),
	steps_per_epoch=len(X_train) // B
	epochs=EPOCHS,
	class_weight=class_weight,
	verbose=1)

[INFO] training network...
Epoch 1/50
4466/4466 [==============================] - 253s 55ms/step - loss: 1.1515 - accuracy: 0.8829 - val_loss: 0.2937 - val_accuracy: 0.9203
Epoch 2/50
4466/4466 [==============================] - 238s 53ms/step - loss: 0.7926 - accuracy: 0.9190 - val_loss: 0.2646 - val_accuracy: 0.9278
Epoch 3/50
4466/4466 [==============================] - 241s 54ms/step - loss: 0.7288 - accuracy: 0.9252 - val_loss: 0.2494 - val_accuracy: 0.9316
Epoch 4/50
4466/4466 [==============================] - 236s 53ms/step - loss: 0.6919 - accuracy: 0.9288 - val_loss: 0.2412 - val_accuracy: 0.9337
Epoch 5/50
4466/4466 [==============================] - 240s 54ms/step - loss: 0.6674 - accuracy: 0.9312 - val_loss: 0.2376 - val_accuracy: 0.9353
Epoch 6/50
4466/4466 [==============================] - 236s 53ms/step - loss: 0.6485 - accuracy: 0.9330 - val_loss: 0.2376 - val_accuracy: 0.9347
Epoch 7/50
4466/4466 [==============================] - 238s 53ms/step - loss: 0.6375 - acc

KeyboardInterrupt: ignored

## RUN 22 EPOCH

In [ ]:
class AccuracyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.998):
      self.model.stop_training = True

    if(logs.get('val_accuracy') > 0.95):
      self.model.stop_training = True

callback = AccuracyCallback()

In [ ]:
from keras.models import load_model
model = load_model('/content/drive/MyDrive/OCR-with-tensorflow-and-opencv-main/ocrrnn_15.hdf5', custom_objects={'categorical_crossentropy': tf.losses.categorical_crossentropy, 'accuracy': tf.keras.metrics.Accuracy})

In [ ]:
print("[INFO] training network...")
H = model.fit(
	aug.flow(X_train, y_train, batch_size=BS),
	validation_data=(X_test, y_test),
	steps_per_epoch=len(X_train) // BS,
	epochs=EPOCHS,
	class_weight=class_weight,
	verbose=1,
	callbacks=[callback])

[INFO] training network...
Epoch 1/7
4466/4466 [==============================] - 255s 53ms/step - loss: 0.8482 - accuracy: 0.9149 - val_loss: 0.2904 - val_accuracy: 0.9219
Epoch 2/7
4466/4466 [==============================] - 235s 53ms/step - loss: 0.8466 - accuracy: 0.9152 - val_loss: 0.2891 - val_accuracy: 0.9222
Epoch 3/7
4466/4466 [==============================] - 236s 53ms/step - loss: 0.8416 - accuracy: 0.9156 - val_loss: 0.2883 - val_accuracy: 0.9220
Epoch 4/7
4466/4466 [==============================] - 235s 53ms/step - loss: 0.8413 - accuracy: 0.9157 - val_loss: 0.2871 - val_accuracy: 0.9227
Epoch 5/7
4466/4466 [==============================] - 234s 52ms/step - loss: 0.8377 - accuracy: 0.9158 - val_loss: 0.2894 - val_accuracy: 0.9217
Epoch 6/7
4466/4466 [==============================] - 234s 52ms/step - loss: 0.8372 - accuracy: 0.9160 - val_loss: 0.2864 - val_accuracy: 0.9227
Epoch 7/7
4466/4466 [==============================] - 266s 59ms/step - loss: 0.8343 - accuracy: 

In [ ]:
model.save('/content/drive/MyDrive/OCR-with-tensorflow-and-opencv-main/ocrrnn_22.hdf5')

## RUN 29 EPOCH

In [ ]:
model2 = load_model('/content/drive/MyDrive/OCR-with-tensorflow-and-opencv-main/ocrrnn_22.hdf5', custom_objects={'categorical_crossentropy': tf.losses.categorical_crossentropy, 'accuracy': tf.keras.metrics.Accuracy})

In [ ]:
print("[INFO] training network...")
hist = model2.fit(
	aug.flow(X_train, y_train, batch_size=BS),
	validation_data=(X_test, y_test),
	steps_per_epoch=len(X_train) // BS,
	epochs=EPOCHS,
	class_weight=class_weight,
	verbose=1,
	callbacks=[callback])

[INFO] training network...
Epoch 1/7
4466/4466 [==============================] - 253s 56ms/step - loss: 0.8329 - accuracy: 0.9165 - val_loss: 0.2851 - val_accuracy: 0.9231
Epoch 2/7
4466/4466 [==============================] - 242s 54ms/step - loss: 0.8306 - accuracy: 0.9165 - val_loss: 0.2847 - val_accuracy: 0.9230
Epoch 3/7
4466/4466 [==============================] - 236s 53ms/step - loss: 0.8298 - accuracy: 0.9168 - val_loss: 0.2850 - val_accuracy: 0.9231
Epoch 4/7
4466/4466 [==============================] - 234s 52ms/step - loss: 0.8274 - accuracy: 0.9171 - val_loss: 0.2835 - val_accuracy: 0.9232
Epoch 5/7
4466/4466 [==============================] - 236s 53ms/step - loss: 0.8269 - accuracy: 0.9167 - val_loss: 0.2831 - val_accuracy: 0.9236
Epoch 6/7
4466/4466 [==============================] - 236s 53ms/step - loss: 0.8259 - accuracy: 0.9171 - val_loss: 0.2831 - val_accuracy: 0.9232
Epoch 7/7
4466/4466 [==============================] - 242s 54ms/step - loss: 0.8227 - accuracy: 

In [ ]:
model2.save('/content/drive/MyDrive/OCR-with-tensorflow-and-opencv-main/ocrrnn_29.hdf5')

## RUN 36 EPOCH

In [ ]:
model3 = load_model('/content/drive/MyDrive/OCR-with-tensorflow-and-opencv-main/ocrrnn_29.hdf5', custom_objects={'categorical_crossentropy': tf.losses.categorical_crossentropy, 'accuracy': tf.keras.metrics.Accuracy})

In [ ]:
print("[INFO] training network...")
hist2 = model3.fit(
	aug.flow(X_train, y_train, batch_size=BS),
	validation_data=(X_test, y_test),
	steps_per_epoch=len(X_train) // BS,
	epochs=EPOCHS,
	class_weight=class_weight,
	verbose=1,
	callbacks=[callback])

[INFO] training network...
Epoch 1/7
4466/4466 [==============================] - 250s 55ms/step - loss: 0.8211 - accuracy: 0.9176 - val_loss: 0.2814 - val_accuracy: 0.9240
Epoch 2/7
4466/4466 [==============================] - 234s 52ms/step - loss: 0.8192 - accuracy: 0.9178 - val_loss: 0.2820 - val_accuracy: 0.9238
Epoch 3/7
4466/4466 [==============================] - 236s 53ms/step - loss: 0.8190 - accuracy: 0.9175 - val_loss: 0.2816 - val_accuracy: 0.9238
Epoch 4/7
4466/4466 [==============================] - 233s 52ms/step - loss: 0.8168 - accuracy: 0.9178 - val_loss: 0.2810 - val_accuracy: 0.9242
Epoch 5/7
4466/4466 [==============================] - 233s 52ms/step - loss: 0.8171 - accuracy: 0.9181 - val_loss: 0.2806 - val_accuracy: 0.9241
Epoch 6/7
4466/4466 [==============================] - 232s 52ms/step - loss: 0.8144 - accuracy: 0.9177 - val_loss: 0.2800 - val_accuracy: 0.9241
Epoch 7/7
4466/4466 [==============================] - 232s 52ms/step - loss: 0.8155 - accuracy: 

In [ ]:
model3.save('/content/drive/MyDrive/OCR-with-tensorflow-and-opencv-main/ocrrnn_36.hdf5')

## run 43 epoch

In [ ]:
model4 = load_model('/content/drive/MyDrive/OCR-with-tensorflow-and-opencv-main/ocrrnn_36.hdf5', custom_objects={'categorical_crossentropy': tf.losses.categorical_crossentropy, 'accuracy': tf.keras.metrics.Accuracy})

In [ ]:
print("[INFO] training network...")
hist3 = model4.fit(
	aug.flow(X_train, y_train, batch_size=BS),
	validation_data=(X_test, y_test),
	steps_per_epoch=len(X_train) // BS,
	epochs=EPOCHS,
	class_weight=class_weight,
	verbose=1,
	callbacks=[callback])

[INFO] training network...
Epoch 1/7
4466/4466 [==============================] - 251s 55ms/step - loss: 0.8135 - accuracy: 0.9181 - val_loss: 0.2808 - val_accuracy: 0.9240
Epoch 2/7
4466/4466 [==============================] - 234s 52ms/step - loss: 0.8105 - accuracy: 0.9184 - val_loss: 0.2799 - val_accuracy: 0.9243
Epoch 3/7
4466/4466 [==============================] - 231s 52ms/step - loss: 0.8115 - accuracy: 0.9185 - val_loss: 0.2788 - val_accuracy: 0.9247
Epoch 4/7
4466/4466 [==============================] - 230s 52ms/step - loss: 0.8111 - accuracy: 0.9180 - val_loss: 0.2787 - val_accuracy: 0.9247
Epoch 5/7
4466/4466 [==============================] - 231s 52ms/step - loss: 0.8084 - accuracy: 0.9188 - val_loss: 0.2778 - val_accuracy: 0.9249
Epoch 6/7
4466/4466 [==============================] - 232s 52ms/step - loss: 0.8100 - accuracy: 0.9189 - val_loss: 0.2777 - val_accuracy: 0.9250
Epoch 7/7
4466/4466 [==============================] - 231s 52ms/step - loss: 0.8081 - accuracy: 

In [ ]:
model4.save('/content/drive/MyDrive/OCR-with-tensorflow-and-opencv-main/ocrrnn_43.hdf5')

## run 50 epoch

In [ ]:
from keras.models import load_model
model5 = load_model('/content/drive/MyDrive/OCR-with-tensorflow-and-opencv-main/ocrrnn_43.hdf5', custom_objects={'categorical_crossentropy': tf.losses.categorical_crossentropy, 'accuracy': tf.keras.metrics.Accuracy})

In [ ]:
print("[INFO] training network...")
hist4 = model5.fit(
	aug.flow(X_train, y_train, batch_size=BS),
	validation_data=(X_test, y_test),
	steps_per_epoch=len(X_train) // BS,
	epochs=EPOCHS,
	class_weight=class_weight,
	verbose=1,
	callbacks=[callback])

[INFO] training network...
Epoch 1/7
4466/4466 [==============================] - 263s 55ms/step - loss: 0.8076 - accuracy: 0.9184 - val_loss: 0.2787 - val_accuracy: 0.9247
Epoch 2/7
4466/4466 [==============================] - 234s 52ms/step - loss: 0.8062 - accuracy: 0.9187 - val_loss: 0.2777 - val_accuracy: 0.9250
Epoch 3/7
4466/4466 [==============================] - 241s 54ms/step - loss: 0.8054 - accuracy: 0.9188 - val_loss: 0.2770 - val_accuracy: 0.9250
Epoch 4/7
4466/4466 [==============================] - 235s 53ms/step - loss: 0.8025 - accuracy: 0.9191 - val_loss: 0.2770 - val_accuracy: 0.9251
Epoch 5/7
4466/4466 [==============================] - 232s 52ms/step - loss: 0.8050 - accuracy: 0.9190 - val_loss: 0.2767 - val_accuracy: 0.9252
Epoch 6/7
4466/4466 [==============================] - 233s 52ms/step - loss: 0.8027 - accuracy: 0.9189 - val_loss: 0.2765 - val_accuracy: 0.9252
Epoch 7/7
4466/4466 [==============================] - 233s 52ms/step - loss: 0.8009 - accuracy: 

In [ ]:
model5.save('/content/drive/MyDrive/OCR-with-tensorflow-and-opencv-main/ocrrnn_50.hdf5')

## Convert model to .bin dan .js

In [ ]:
pip install tensorflowjs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 77 kB 5.8 MB/s 
     |████████████████████████████████| 40 kB 7.1 MB/s 
     |████████████████████████████████| 462 kB 34.9 MB/s 
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!tensorflowjs_converter --input_format=keras {'/content/drive/MyDrive/OCR-with-tensorflow-and-opencv-main/ocrrnn_50.hdf5'} ./

## Evaluate Model

In [ ]:
# define the list of label names
labelNames = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]
# evaluate the network
print("[INFO] evaluating network...")
predictions = model5.predict(X_test, batch_size=BS)
print(classification_report(y_test.argmax(axis=1),
	predictions.argmax(axis=1), target_names=labelNames))

[INFO] evaluating network...
              precision    recall  f1-score   support

           A       0.93      0.92      0.92      7215
           B       0.92      0.91      0.91      5915
           C       0.97      0.93      0.95      9600
           D       0.89      0.90      0.89      6281
           E       0.94      0.92      0.93      6608
           F       0.71      0.92      0.80      4038
           G       0.86      0.91      0.89      5188
           H       0.84      0.91      0.87      5552
           I       0.74      0.87      0.80      4027
           J       0.93      0.91      0.92      5871
           K       0.89      0.92      0.90      5148
           L       0.96      0.92      0.94      6644
           M       0.93      0.93      0.93      6832
           N       0.94      0.93      0.94      8500
           O       0.99      0.93      0.96     18204
           P       0.98      0.94      0.96      8583
           Q       0.82      0.91      0.86      520